In [1]:
class Graph:
    def __init__(self, directed=False):
        self.graph = {}
        self.directed = directed
    
    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []
    
    def add_edge(self, src, dest):
        if src not in self.graph:
            self.add_vertex(src)
        if dest not in self.graph:
            self.add_vertex(dest)
        self.graph[src].append(dest)
        if not self.directed:
            self.graph[dest].append(src)
    
    def remove_edge(self, src, dest):
        if src in self.graph:
            if dest in self.graph[src]:
                self.graph[src].remove(dest)
        if not self.directed:
            if dest in self.graph and src in self.graph[dest]:
                self.graph[dest].remove(src)
    
    def remove_vertex(self, vertex):
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    self.graph[adj].remove(vertex)
            # Remove the vertex entry
            del self.graph[vertex]
    
    def get_adjacent_vertices(self, vertex):
        if vertex in self.graph:
            return self.graph[vertex]
        else:
            return []
    
    def __str__(self):
        return str(self.graph)

# Example usage:
g = Graph(directed=True)
g.add_vertex('A')
g.add_vertex('B')
g.add_edge('A', 'B')
g.add_edge('A', 'C')
print(g)


{'A': ['B', 'C'], 'B': [], 'C': []}


In [2]:
import threading
from collections.abc import Hashable
from typing import Dict, Set, List

class Graph:
    def __init__(self, directed: bool = False):
        self._graph: Dict[Hashable, Set[Hashable]] = {}
        self.directed = directed
        self._lock = threading.Lock()

    def _validate_vertex(self, v: Hashable) -> None:
        # Ensure the vertex is hashable and of allowed type
        if not isinstance(v, Hashable):
            raise TypeError("Vertex must be a hashable type")

    def add_vertex(self, vertex: Hashable) -> None:
        self._validate_vertex(vertex)
        with self._lock:
            self._graph.setdefault(vertex, set())

    def add_edge(self, src: Hashable, dest: Hashable) -> None:
        self._validate_vertex(src)
        self._validate_vertex(dest)
        with self._lock:
            # Ensure both vertices exist
            self._graph.setdefault(src, set())
            self._graph.setdefault(dest, set())
            # Add edge
            self._graph[src].add(dest)
            if not self.directed:
                self._graph[dest].add(src)

    def remove_edge(self, src: Hashable, dest: Hashable) -> None:
        with self._lock:
            # Safely discard edges if present
            if src in self._graph:
                self._graph[src].discard(dest)
            if not self.directed and dest in self._graph:
                self._graph[dest].discard(src)

    def remove_vertex(self, vertex: Hashable) -> None:
        with self._lock:
            if vertex in self._graph:
                # Remove incoming edges
                for neighbors in self._graph.values():
                    neighbors.discard(vertex)
                # Remove the vertex itself
                del self._graph[vertex]

    def get_adjacent_vertices(self, vertex: Hashable) -> List[Hashable]:
        with self._lock:
            # Return a copy to prevent external modification
            return list(self._graph.get(vertex, set()))

    def __str__(self) -> str:
        with self._lock:
            # Return a shallow copy for safe printing
            return str({v: set(neighbors) for v, neighbors in self._graph.items()})

# Example usage:
if __name__ == "__main__":
    g = Graph(directed=True)
    g.add_vertex('A')
    g.add_vertex('B')
    g.add_edge('A', 'B')
    g.add_edge('A', 'C')
    print(g)  # {'A': {'B', 'C'}, 'B': set(), 'C': set()}


{'A': {'C', 'B'}, 'B': set(), 'C': set()}


In [3]:
class Graph:
    def __init__(self, directed=False):
        """
        Initialize the Graph.

        Parameters:
        - directed (bool): Specifies whether the graph is directed. Default is False (undirected).

        Attributes:
        - graph (dict): A dictionary to store vertices and their adjacent vertices.
        - directed (bool): Indicates whether the graph is directed.
        """
        self.graph = {}
        self.directed = directed
    
    def add_vertex(self, vertex):
        """
        Add a vertex to the graph.

        Parameters:
        - vertex: The vertex to add. It must be hashable.

        Ensures that each vertex is represented in the graph dictionary as a key with an empty list as its value.
        """
        if not isinstance(vertex, (int, str, tuple)):
            raise ValueError("Vertex must be a hashable type.")
        if vertex not in self.graph:
            self.graph[vertex] = []
    
    def add_edge(self, src, dest):
        """
        Add an edge from src to dest. If the graph is undirected, also add from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.

        Prevents adding duplicate edges and ensures both vertices exist.
        """
        if src not in self.graph or dest not in self.graph:
            raise KeyError("Both vertices must exist in the graph.")
        if dest not in self.graph[src]:  # Check to prevent duplicate edges
            self.graph[src].append(dest)
        if not self.directed and src not in self.graph[dest]:
            self.graph[dest].append(src)
    
    def remove_edge(self, src, dest):
        """
        Remove an edge from src to dest. If the graph is undirected, also remove from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        """
        if src in self.graph and dest in self.graph[src]:
            self.graph[src].remove(dest)
        if not self.directed and dest in self.graph and src in self.graph[dest]:
            self.graph[dest].remove(src)
    
    def remove_vertex(self, vertex):
        """
        Remove a vertex and all edges connected to it.

        Parameters:
        - vertex: The vertex to be removed.
        """
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    self.graph[adj].remove(vertex)
            # Remove the vertex entry itself
            del self.graph[vertex]
    
    def get_adjacent_vertices(self, vertex):
        """
        Get a list of vertices adjacent to the specified vertex.

        Parameters:
        - vertex: The vertex whose neighbors are to be retrieved.

        Returns:
        - List of adjacent vertices. Returns an empty list if vertex is not found.
        """
        return self.graph.get(vertex, [])
    
    def __str__(self):
        """
        Provide a string representation of the graph's adjacency list for easy printing and debugging.

        Returns:
        - A string representation of the graph dictionary.
        """
        return str(self.graph)

# Example usage:
try:
    g = Graph(directed=True)
    g.add_vertex('A')
    g.add_vertex('B')
    g.add_edge('A', 'B')
    g.add_edge('A', 'B')  # Attempt to add duplicate edge
    print(g)
except Exception as e:
    print(f"Error: {e}")


{'A': ['B'], 'B': []}
